In [9]:
import numpy as np
import pylab as plt
import sugar
import copy
import sugar_generator as sg
%matplotlib inline
model = np.loadtxt('../sugar_model/SUGAR_model_v1.asci')

In [127]:
def get_a_sed(model, q1, q2, q3, Av, grey):
    reorder = np.arange(197*21).reshape(21, 197).T.reshape(-1)
    sed = np.zeros_like(model[:,2])
    sed += copy.deepcopy(model[:,2])
    sed += q1 * copy.deepcopy(model[:,3])
    sed += q2 * copy.deepcopy(model[:,4])
    sed += q3 * copy.deepcopy(model[:,5])
    sed += Av * copy.deepcopy(model[:,6])
    sed += grey
    return sed

francis = get_a_sed(model, 1., 1., 1., 0.1, 0.)

In [128]:
def flbda2fnu( x, y, var=None, backward=False):
    """Convert *x* [A], *y* [erg/s/cm2/A] to *y* [erg/s/cm2/Hz]. Se
    `var=var(y)` to get variance."""

    f = x**2 / 299792458. * 1.e-10 # Conversion factor                                                                                                                      
    if backward: 
        f = 1./f   
    if var is None:                # Return converted signa
        return y * f
    else:                          # Return converted variance
        return var * f**2

def flbda2ABmag( x, y, ABmag0=48.59, var=None):
    """Convert *x* [A], *y* [erg/s/cm2/A] to `ABmag =                                                                                                                       
    -2.5*log10(erg/s/cm2/Hz) - ABmag0`. Set `var=var(y)` to get                                                                                                             
    variance."""          
    z = flbda2fnu(x,y)
    if var is None:
        return -2.5*np.log10(z) - ABmag0 
    else:
        return (2.5/np.log(10)/z)**2 * flbda2fnu(x,y,var=var)

In [129]:
def get_param(sed, sed_err, model):
    reorder = np.arange(197*21).reshape(21, 197).T.reshape(-1)
    W = np.eye(len(sed_err))*1./sed_err
    A = np.array([np.ones_like(model[:,3]), model[:,3],model[:,4], model[:,5], model[:,6]]).T
    left = np.linalg.inv(np.dot(A.T,np.dot(W, A)))
    right = np.dot(A.T, np.dot(W, sed - model[:,2]))
    h = np.dot(left, right)
    print h 
    
get_param(francis, np.ones_like(francis), model)

[  1.55431223e-15   1.00000000e+00   1.00000000e+00   1.00000000e+00
   1.00000000e-01]


In [130]:
ss = sg.sugar_spectrum()
ss._parameters = np.array([1., 1., 1., 0.1, 0.])
spec_mag = []
spec_mag_err = []
reorder = np.arange(197*21).reshape(21, 197).T.reshape(-1)
#ss._wave = np.sort(model[:,1][:197])
print model[:,0][4136], model[:,1][4136]

for p in (ss._phase):
    spec_mag_p = flbda2ABmag(ss._wave, ss.model_spectrum_flux_m0(p, ss._wave)[0])  
    
    spec_mag_p_err = np.ones_like(spec_mag_p)*0.000000000000001
    spec_mag += list(spec_mag_p)
    spec_mag_err += list(spec_mag_p_err)
spec_mag = np.array(spec_mag)
spec_mag_err = np.array(spec_mag_err)
print ss._phase, ss._wave
print spec_mag[reorder]
print model[:,2]+ model[:,3]+ model[:,4]+ model[:,5]+ model[:,6]*0.1

48.0 8649.03871
[-12.  -9.  -6.  -3.   0.   3.   6.   9.  12.  15.  18.  21.  24.  27.  30.
  33.  36.  39.  42.  45.  48.] [ 3254.01639  3270.28648  3286.63791  3303.0711   3319.58645  3336.18439
  3352.86531  3369.62963  3386.47778  3403.41017  3420.42722  3437.52936
  3454.717    3471.99059  3489.35054  3506.7973   3524.33128  3541.95294
  3559.6627   3577.46102  3595.34832  3613.32506  3631.39169  3649.54865
  3667.79639  3686.13537  3704.56605  3723.08888  3741.70432  3760.41284
  3779.21491  3798.11098  3817.10154  3836.18705  3855.36798  3874.64482
  3894.01805  3913.48814  3933.05558  3952.72085  3972.48446  3992.34688
  4012.30862  4032.37016  4052.53201  4072.79467  4093.15864  4113.62444
  4134.19256  4154.86352  4175.63784  4196.51603  4217.49861  4238.5861
  4259.77903  4281.07793  4302.48332  4323.99573  4345.61571  4367.34379
  4389.18051  4411.12641  4433.18204  4455.34795  4477.62469  4500.01282
  4522.51288  4545.12545  4567.85107  4590.69033  4613.64378  4636.712
  4

In [131]:
get_param(spec_mag[reorder], np.ones_like(francis), model)


[  3.10862447e-15   1.00000000e+00   1.00000000e+00   1.00000000e+00
   1.00000000e-01]
